# Race Condition Example

**Imports**

In [1]:
from threading import Thread
import os
import time

**Our global variable: May be stored in database**

In [2]:
balance = 50

**Methods Performing Transaction:**
- Note that we're simulating the IO operation with sleep before saving to global state.

In [3]:
def increase_balance(amount):
    """Increases the balance by passed amout"""
    global balance
    bl = balance # local copy
    bl += amount # increase balance by amount
    
    time.sleep(2) # wait / perform IO operation
    
    balance = bl # write to global state

def decrease_balance(amount):
    """Decreases the balance by passed amout"""
    global balance
    bl = balance # local copy
    bl -= amount # decrease balance by amount
    
    time.sleep(2) # wait / perform IO operation
    
    balance = bl # write to global state

**Initialize the threads:**
- Each thread increases / decreases the balance by 100.

In [4]:
inc_thread = Thread(target=increase_balance, args=(100, )) # increase balance by 100
dec_thread = Thread(target=decrease_balance, args=(100,)) # decrease balance by 100

**Start Threads:**

In [5]:
inc_thread.start()
dec_thread.start()

**Join**
- Block main thread untill completion of those two threads.

In [6]:
inc_thread.join()
dec_thread.join()

print(f"Balance after transaction is: {balance}")

Balance after transaction is: -50


**What's the issue here ?**
- Here we've 2 threads, each increasing and decreasing the balance by 100. Initial balance is 50.
- **Expected Result:** ``50+100-100=50``
- **Result We're getting:** ``-50``

**Why is there's a incostitent balance ?**

- While one of the thread (which might be performing increament or decreament ) was sleeping for 2 seconds (performing) IO tasks.

- As a thread waiting for IO (sleep) goes into sleep other thread arrived to the point and does the operation.

- A **potential race condition.**

# Solution
- Using mutex / lock.
- What lock does is, it allows the critical section to be accessed by only one single thread.

**Imports**

In [7]:
from threading import Lock

In [8]:
def increase_balance(amount, lock):
    """Increases the balance by passed amout"""
    global balance
    
    lock.acquire() # Acquire lock
    
    bl = balance # local copy
    bl += amount # increase balance by amount
    time.sleep(2) # wait / perform IO operation
    balance = bl # write to global state
    
    lock.release() # Release lock

def decrease_balance(amount, lock):
    """Decreases the balance by passed amout"""
    global balance
    
    # lock as a context
    with lock:  
        bl = balance # local copy
        bl -= amount # decrease balance by amount
        time.sleep(2) # wait / perform IO operation
        balance = bl # write to global state
    

In [9]:
balance = 50

**Initialize the Lock**

In [10]:
lock = Lock()

In [12]:
inc_thread = Thread(target=increase_balance, args=(100, lock)) # increase balance by 100
dec_thread = Thread(target=decrease_balance, args=(100, lock)) # decrease balance by 100

inc_thread.start()
dec_thread.start()

inc_thread.join()
dec_thread.join()

print(f"Balance after transaction is: {balance}")

Balance after transaction is: 50
